In [1]:
import sys, os

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from debugger.tree_node import NodeRef
from evaluator import BenchViewer, Evaluator
from debugger.debugger import Debugger3
from evaluator import DebuggerStatus
from benchmark_consts import *
from utils.system_utils import list_smt2_files, get_name_hash

In [2]:
qs = UNSTABLE_MARIPOSA
mv = BenchViewer(qs)
# vv = BenchViewer(UNSTABLE_VERUS)

[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proof available 
[WARN] [eval] be808dd0c3 tested count 997 < registered 998 


[ERROR] no such project group singleton_b1b86518f3 


[WARN] [eval] 9b9e796545 tested count 1076 < registered 1079 
[WARN] [eval] 0e45874c62 tested count 991 < registered 993 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proof available 
[WARN] [eval] cc5b8bed53 tested count 289 < registered 497 
[WARN] [eval] 09f0c73516 tested count 398 < registered 972 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proof available 
[WARN] [eval] d8cf640f87 tested count 1043 < registered 1044 


[ERROR] no such project group singleton_563f5abfa4 [ERROR] no such project group singleton_ee8d1ab295 

[ERROR] no such project group singleton_90e97b47ee 


[WARN] [eval] e695dc7eeb tested count 1159 < registered 1160 
[WARN] [eval] 519fdc89c1 tested count 506 < registered 508 


[ERROR] no such project group singleton_e93855aadd 


[WARN] [eval] 203627efb4 tested count 6 < registered 404 


[ERROR] no such project group singleton_f6db5998ec 
[ERROR] no such project group singleton_d874a82c3a 
[ERROR] no such project group singleton_0b72dcef64 


[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proof available 


[ERROR] no such project group singleton_ab52dadac1 


In [3]:
mv.status.print_status()

| category                                      |   count | percentage   |
|-----------------------------------------------|---------|--------------|
| DebuggerStatus.FINISHED                       |     523 | 95.96 %      |
| DebuggerStatus.SINGLETON_CREATION_FAILED      |       9 | 1.65 %       |
| DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED |       9 | 1.65 %       |
| DebuggerStatus.NO_PROOF                       |       4 | 0.73 %       |
| total                                         |     545 | 100.00 %     |


In [ ]:
for q in mv.status[DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED]:
    print("./src/debugger3.py --create-singleton -i", q)

for q in mv.status[DebuggerStatus.SINGLETON_CREATION_FAILED]:
    print("./src/debugger3.py --create-singleton -i", q)

In [ ]:
for q in mv.status[DebuggerStatus.SINGLETON_NOT_RAN]:
# for q in mv.status[DebuggerStatus.SINGLETON_TESTED_EMPTY]:
    assert len(list_smt2_files(mv[q].singleton_dir)) != 0
    print("./src/exper_wizard.py manager -e verify --total-parts 12 -i ", mv[q].singleton_dir, "--clear")
    print("./src/analysis_wizard.py singleton -e verify -s z3_4_13_0 -i ", mv[q].singleton_dir)
    print("python3 src/carve_and_rerun.py", mv[q].filtered_dir)

In [ ]:
import numpy as np
# print(len(mv.fixable)/len(mv.status[DebuggerStatus.FINISHED]))

bad_min_ranks = dict()
min_ranks = []

for q in mv.fixable:
    r = mv.reviewers[q]
    report = r.build_report()
    indices = report.freq.loc[report.freq["qname"].isin(report.stabilized["qname"])].index
    report.freq["rank"] = report.freq["trace_count"].rank(method='min', ascending=False)
    min_rank = report.freq.loc[indices]["rank"].min()
    max_rank = report.freq.loc[indices]["rank"].max()
    min_ranks.append(min_rank)
    if min_rank <= 10:
        continue
    bad_min_ranks[q] = (min_rank, len(indices))

min_ranks = np.array(min_ranks)

for q, (rank, feasible) in bad_min_ranks.items():
    print(int(rank), feasible, q)

In [ ]:
def fmt_ratio(x, y):
    return f"({round(x / y * 100, 1)}%)"

def fmt_ratio_2(x, y):
    return f"{x}/{y} ({round(x / y * 100, 1)}%)"

def count_leq(r, n):
    return np.where(r <= n)[0].shape[0]

fixable_count = len(mv.fixable)
finished_count = len(mv.status[DebuggerStatus.FINISHED])
bench_total = mv.status.total
print("finished", fmt_ratio_2(finished_count, bench_total))
print("\tfixable", fmt_ratio_2(fixable_count, finished_count))
rcount = count_leq(min_ranks, 1)
print("\t\t", "top-1 ", fmt_ratio_2(rcount, fixable_count))
rcount = count_leq(min_ranks, 3)
print("\t\t", "top-3 ", fmt_ratio_2(rcount, fixable_count))
rcount = count_leq(min_ranks, 10)
print("\t\t", "top-10", fmt_ratio_2(rcount, fixable_count))
print("\tno-fixes", fmt_ratio_2(finished_count - fixable_count, finished_count))
no_proof = len(mv.status[DebuggerStatus.NO_PROOF])
print("no-proof", fmt_ratio_2(no_proof, bench_total))
parse_fail = len(mv.status[DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED]) + len(mv.status[DebuggerStatus.SINGLETON_CREATION_FAILED])
print("parse-fail", fmt_ratio_2(parse_fail, bench_total))
in_progress = bench_total - finished_count - parse_fail - no_proof
print("in-progress", fmt_ratio_2(in_progress, bench_total))

In [ ]:
for q in mv.status[DebuggerStatus.NO_PROOF]:
    print('"' + mv[q].name_hash + '"', end=", ")

In [ ]:
dirs_to_send = []
additional = ""

for q in mv.fixable:
    r = mv.reviewers[q]
    if r.name_hash in LIKELY_FIXED:
        continue
    r.collect_garbage()
    dirs_to_send += r.get_dirs_to_sync()
    additional += f"\"{r.name_hash}\",\n"

print(additional)

for i, d in enumerate(dirs_to_send):
    if i % 5 == 0:
        print("echo '", (i //5 +1), "/", len(dirs_to_send)//5, "'")
    print("rsync -avz", d, "g2001:/home/yizhou7/mariposa/", "--inplace", "--delete", "--relative")

In [4]:
from utils.analysis_utils import Categorizer

f_modes = Categorizer()

for query in mv.unfixable:
    rev: Evaluator = mv[query]
    mi = rev.get_trace_info()
    reason = mi.get_failed_reason()
    f_modes.add_item(reason, query)

f_modes.finalize()

In [5]:
f_modes.print_status()

| category                  |   count | percentage   |
|---------------------------|---------|--------------|
| TraceFailure.TIMEOUT      |     116 | 82.27 %      |
| TraceFailure.FAST_UNKNOWN |      22 | 15.6 %       |
| TraceFailure.SLOW_UNKNOWN |       3 | 2.13 %       |
| total                     |     141 | 100.00 %     |


In [ ]:
from debugger.mutant_info import TraceFailure

def double_check_tested(ev: Evaluator):
    # mi = ev.get_trace_info()
    # trace_size = mi.get_trace_size()
    # if trace_size > 300 * 1024 * 1024:
    #     return (trace_size, ev.name_hash, mi.trace_path)
    actions = ev.editor.get_singleton_actions()
    expected = len(actions)
    tested = len(ev.build_report().tested)
    if tested != expected:
        return (expected, tested, ev.name_hash)
    return None

# for q in f_modes[TraceFailure.TIMEOUT]:
#     ev = mv[q]

from multiprocessing import Pool
pool = Pool(6)

res = pool.map(double_check_tested, [mv[q] for q in mv.unfixable])
res = [r for r in res if r is not None]

[DEBUG] [edit] proof path: dbg/a05a8d0e07/proofs/shuffle.8479680377847396330.proof 
[DEBUG] [edit] trace path: dbg/a05a8d0e07/traces/rename.10513464605750370432 


252615f283037ed8e2112ab12df8355a.pickle
[WARN] [differ] qid constructor_accessor_axiom not found in dbg/0cce886818/orig.smt2 
[WARN] [differ] qid <null> not found in dbg/0cce886818/orig.smt2 


[DEBUG] [edit] proof path: dbg/b837e52d8b/proofs/rename.5155518270293062284.proof 
[DEBUG] [edit] trace path: dbg/b837e52d8b/traces/shuffle.11548164802347118094 


70c0ac9aca7ad99f60b790b547e61ea5.pickle
[WARN] [differ] qid constructor_accessor_axiom not found in dbg/68aa54b124/orig.smt2 


[DEBUG] [edit] proof path: dbg/751af3ffba/proofs/rename.6841123210129246270.proof 
[DEBUG] [edit] trace path: dbg/751af3ffba/traces/rename.11153525741169455601 


...

22bf960de458e1a25185d95efc8cc2cf.pickle
[WARN] [differ] qid constructor_accessor_axiom not found in dbg/9967bcf803/orig.smt2 


[DEBUG] [edit] proof path: dbg/50e9564b05/proofs/reseed.3435255191336675210.proof 
[DEBUG] [edit] trace path: dbg/50e9564b05/traces/rename.8782054682830391215 


8b3d849720f1197ad62d735be4e60350.pickle
[WARN] [differ] qid constructor_accessor_axiom not found in dbg/0711a2f033/orig.smt2 
[WARN] [differ] qid <null> not found in dbg/0711a2f033/orig.smt2 


[DEBUG] [edit] proof path: dbg/01b00606af/proofs/reseed.16567756780824535738.proof 
[DEBUG] [edit] trace path: dbg/01b00606af/traces/rename.17488541728259935425 


8e876d6d248844e32573419ddef0f34e.pickle
[WARN] [differ] qid constructor_accessor_axiom not found in dbg/50e9564b05/orig.smt2 
[WARN] [differ] qid <null> not found in dbg/50e9564b05/orig.smt2 
[WARN] [differ] qid constructor_accessor_axiom not found in dbg/01b00606af/orig.smt2 


In [14]:
for r in res:
    print(r)

({'mariposa_qid_361': <EditAction.ERASE: 'erase'>, 'mariposa_qid_3187': <EditAction.ERASE: 'erase'>, 'mariposa_qid_215': <EditAction.ERASE: 'erase'>, 'mariposa_qid_4638': <EditAction.ERASE: 'erase'>, 'mariposa_qid_2901': <EditAction.INST_REPLACE: 'inst_replace'>, 'mariposa_qid_2906': <EditAction.INST_REPLACE: 'inst_replace'>, 'mariposa_qid_3799': <EditAction.ERASE: 'erase'>, 'mariposa_qid_2903': <EditAction.ERASE: 'erase'>, 'mariposa_qid_3186': <EditAction.ERASE: 'erase'>, 'mariposa_qid_206': <EditAction.ERASE: 'erase'>, 'mariposa_qid_339': <EditAction.INST_REPLACE: 'inst_replace'>, 'mariposa_qid_396': <EditAction.ERASE: 'erase'>, 'mariposa_qid_2970': <EditAction.INST_REPLACE: 'inst_replace'>, 'mariposa_qid_5337': <EditAction.ERASE: 'erase'>, 'mariposa_qid_5333': <EditAction.ERASE: 'erase'>, 'mariposa_qid_1504': <EditAction.ERASE: 'erase'>, 'mariposa_qid_4983': <EditAction.ERASE: 'erase'>, 'mariposa_qid_381': <EditAction.INST_KEEP: 'inst_keep'>, 'mariposa_qid_2137': <EditAction.INST_RE